https://www.kaggle.com/datasets/hojjatk/mnist-dataset

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


data = pd.read_csv(r'../../../2. MNIST_dataset\train.csv')
data.sample(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
13708,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1200,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4376,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
data.iloc[0]

label       1
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Name: 0, Length: 785, dtype: int64

In [25]:
data.shape

(42000, 785)

In [26]:
data = np.array(data)
data

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [7, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0]])

In [27]:
m, n = data.shape
m, n

(42000, 785)

In [28]:
np.random.shuffle(data)
data

array([[6, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [7, 0, 0, ..., 0, 0, 0],
       ...,
       [8, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [29]:
# picking only 1000 examples for training
data_dev = data[:1000, :]

# transposing data, so that features lie down horizontally, as we visualize the input of a NN architecture.
data_dev = data_dev.T
data_dev.shape

(785, 1000)

In [30]:
data_dev

array([[6, 4, 7, ..., 4, 8, 5],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [31]:
y_dev = data_dev[0]
x_dev = data_dev[1:]

In [32]:
data_train = data[10000:].T
y_train = data_train[0]
x_train = data_train[1:]

In [33]:
def init_params():
    h1_w1 = np.random.randn(10, 784) - 0.5
    h1_b1 = np.random.randn(10, 1) - 0.5

    h2_w2 = np.random.randn(10, 10) - 0.5
    h2_b2 = np.random.randn(10, 1) - 0.5

    return h1_w1, h1_b1, h2_w2, h2_b2

def relu(z):
    return np.maximum(0, z)

def softmax(z):
    return np.exp(z) / np.sum(np.exp(z))

def forward_prop(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1
    a1 = relu(z1)
    z2 = w2.dot(a1) + b2
    a2 = softmax(a1)
    return z1, a1, z2, a2

def one_hot(y):
    one_hot_y = np.zeros((y.size, y.max()+1))
    one_hot_y[np.arange(y.size), y] = 1
    one_hot_y = one_hot_y.T
    return one_hot_y

def deriv_relu(z):
    return z>0

def back_prop(z1, a1, z2, a2, w2, x, y):
    m = y.size
    one_hot_y = one_hot(y)
    dz2 = a2 - one_hot_y
    dw2 = 1/ m*dz2.dot(a1.T)
    db2 = 1/ m*np.sum(dz2, 1)
    dz1 = w2.T.dot(dz2)*deriv_relu(z1)
    dw1 = 1/ m*dz1.dot(x.T)
    db1 = 1/ m*np.sum(dz1, 1)
    return dw1, db1, dw2, db2

def update_param(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    w1 = w1 - alpha*dw1
    b1 = b1 - alpha*db1
    w2 = w2 - alpha*dw2
    b2 = b2 - alpha*db2
    return w1, b1, w2, b2

def get_prediction(a2):
    return np.argmax(a2, 0)

def get_accuracy(predictions, y):
    print(predictions, y)
    return np.sum(predictions == y) / y.size

def gradient_descent(x, y, iterations, alpha):
    w1, b1, w2, b2 = init_params()
    for i in range(iterations):
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, x)
        dw1, db1, dw2, db2 = back_prop(z1, a1, z2, a2, w2, x, y)
        w1, b1, w2, b2 = update_param(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        if i%50 == 0:
            print('Iteration: ', i)
            print('Accuracy: ', get_accuracy(get_prediction(a2), y))
    return w1, b1, w2, b2


In [35]:
w1, b1, w2, b2 = gradient_descent(x_train, y_train, 100, 0.1)

Iteration:  0
[0 0 0 ... 0 0 0] [0 0 9 ... 8 1 0]
Accuracy:  0.098125


ValueError: operands could not be broadcast together with shapes (10,32000) (10,10) 